## Logic to Karnaugh map

[Wiki: Karnaugh map](https://en.wikipedia.org/wiki/Karnaugh_map)
![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/K-map_6%2C8%2C9%2C10%2C11%2C12%2C13%2C14_anti-race.svg/440px-K-map_6%2C8%2C9%2C10%2C11%2C12%2C13%2C14_anti-race.svg.png)

In [8]:
from sympy.combinatorics.graycode import GrayCode
from itertools import permutations
import numpy as np

In [9]:
def ReverseBinCode(l):
    # Generate reversed binary code for pyphi
    maxIndex = 2 ** l - 1
    return [np.binary_repr(maxIndex - index, width=l) for index in range(2 ** l)]

## 输入`pyphi`中状态转移矩阵，给出各个变量的卡诺图

In [10]:
# 输入状态转移列表，给出各个量的卡诺图矩阵

def MapToKarnaughMap(_map):
    # Turn map to MapToKarnaugh Maps
    # Generate ReverseBinCode
    index = ReverseBinCode(len(_map[0]))
    #index = np.array([[int(char) for char in strcode] for strcode in index])
    
    dim1 = (int(len(_map[0]) / 2))
    dim2 = (len(_map[0]) - int(len(_map[0]) / 2))
    #print('dim =', dim1, 'x', dim2)
    
    gc1 = GrayCode(dim1)
    gc2 = GrayCode(dim2)
    gd1 = dict(zip(list(gc1.generate_gray()), range(2 ** dim1)))
    gd2 = dict(zip(list(gc2.generate_gray()), range(2 ** dim2)))
    #print(gd1)
    #print(gd2)
    
    _map = np.array(_map).T
    KarnaughMaps = []
    for targetStates in _map:
        #print('-------')
        Karnaugh = np.array(([[0] * (2 ** dim1)]) * (2 ** dim2)).T
        #print(Karnaugh)
        for state, mcode in zip(index, targetStates):
            #print(state, '->', mcode)
            #print('c1=',state[0:dim1])
            #print('c2=',state[dim1:])
            Karnaugh[gd1[state[0:dim1]], gd2[state[dim1:]]] = mcode
        KarnaughMaps += [Karnaugh]
        #print(Karnaugh)
    
    return np.array(KarnaughMaps)

## 输入矩阵，给出Ising模型能量

In [11]:
def IsingEnergy(mat, energy={0:-1, 1:1}):
    # 给出mat的Ising能量
    # Return Ising energy of mat
    mat = np.array(mat)
    # If mat is a list of mat, then return all energy
    if len(mat.shape) == 3:
        return np.array([IsingEnergy(_mat, energy) for _mat in mat])
    # Otherwise return this energy
    x, y = mat.shape
    total_E = 0
    for i in range(x):
        for j in range(y):
            total_E += energy[mat[i][j]] * energy[mat[(i - 1) % x][j]]
            total_E += energy[mat[i][j]] * energy[mat[(i + 1) % x][j]]
            total_E += energy[mat[i][j]] * energy[mat[i][(j - 1) % y]]
            total_E += energy[mat[i][j]] * energy[mat[i][(j + 1) % y]]
    return -total_E

In [5]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]
MapToKarnaughMap(m)

array([[[1, 1, 1, 1],
        [0, 0, 0, 0]],

       [[1, 1, 0, 0],
        [1, 1, 0, 0]],

       [[1, 0, 0, 1],
        [1, 0, 0, 1]]])

## 对状态名重新标记 Relabel

In [7]:
testorder=permutations(m).__next__()

In [8]:
print(relabel(m, testorder))

[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]


In [9]:
MapToKarnaughMap(relabel(testM, testOrder))

NameError: name 'testM' is not defined

## 计算不同排序的Ising能量

In [54]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]
energyList = []
maxlen = 50

testorders=permutations(m)

for i, order in zip(range(maxlen), testorders):
    current_m = relabel(m, list(order))
    energy = [IsingEnergy(kmap) for kmap in MapToKarnaughMap(current_m)]
    energyList += energy

print(energyList)

[0, -16, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 8, 8, 0, 32, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 0, -16, 0, 16, 16, 0, 8, 8, 0, 8, 8, 0, 16, 16, 0, 32, -16, 0, 8, 8, 0, -16, -16, 0, -16, 0, 0, 0, -16, 0, 8, 8, 0, 8, 8, 0, -16, -16, 0, -16, 0, 0, -16, -16, 0, -16, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 8, 8, 0, 32, -16, -16, 0, -16, -16, 0, 32, 0, 0, 0, 8, 0, 8, 0, 16, 16, 8, 16, -16, 0, 0, 0, 0, 16, 16, -16, -16, 0, 8, -16, 16, -16, 32, 0, 8, 8, 0, 8, 0, 8, 8, 16, -16, 8, -16, 16, 32, -16, -16, 8, 8, 0, 32, -16, -16, 0, -16, 32, 0, -16, -16]


## NN for learning dynamic of network

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [15]:
class net(nn.Module):
    def __init__(self, size_vis, size_hidden):
        super(net, self).__init__()
        self.fcn_v = nn.Linear(size_vis, size_vis)
        self.fcn_v2h = nn.Linear(size_vis, size_hidden)
        self.fcn_h2v = nn.Linear(size_hidden, size_vis)
        
    def forward(self, x):
        h = torch.sigmoid(self.fcn_v2h(x))
        xp = torch.sigmoid(self.fcn_v(x))
        
        xout = torch.sigmoid(self.fcn_h2v(h) + self.fcn_v(xp))
        
        return xout

In [12]:
IITnet = net(3, 4)

In [13]:
IITnet.forward(torch.Tensor([0,0,0]))

tensor([0.5867, 0.4844, 0.2930], grad_fn=<SigmoidBackward>)

In [28]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]

In [16]:
def BinCodeList(length):
    s = ReverseBinCode(length)

    return [[int(char) for char in ss] for ss in s]

class graphData(data.Dataset):
    def __init__(self, _m):
        super(graphData, self).__init__()
        self.inputs = BinCodeList(len(m[0]))
        self.m = _m
        
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, index):
        return self.inputs[index], self.m[index]

In [93]:
IITnet = net(3, 4)



dataset = graphData(m)
setLoader = data.DataLoader(dataset, batch_size = 1)
optimizer = optim.SGD(IITnet.parameters(), lr = 0.01, momentum=0.9, weight_decay=0.0005)

loss_fn = torch.nn.MSELoss()

for epco in range(4000):
    for inputdata, target in setLoader:
        optimizer.zero_grad()
        output = IITnet.forward(torch.Tensor(inputdata))
        loss = loss_fn(output, torch.Tensor(target))
        loss.backward()
        optimizer.step()
    if epco % 400 == 0:
        print('epco', epco, 'loss:',loss.item())

epco 0 loss: 0.22029556334018707
epco 400 loss: 0.08167281746864319
epco 800 loss: 0.021813470870256424
epco 1200 loss: 0.014217870309948921
epco 1600 loss: 0.012218523770570755
epco 2000 loss: 0.011513054370880127
epco 2400 loss: 0.011218958534300327
epco 2800 loss: 0.011076707392930984
epco 3200 loss: 0.010997287929058075
epco 3600 loss: 0.010946993716061115


In [17]:
def trainNet(_m):
    _IITnet = net(3, 4)

    dataset = graphData(_m)
    setLoader = data.DataLoader(dataset, batch_size = 1)
    optimizer = optim.SGD(_IITnet.parameters(), lr = 0.01, momentum=0.9, weight_decay=0.0005)

    loss_fn = torch.nn.MSELoss()

    for epco in range(1400):
        for inputdata, target in setLoader:
            optimizer.zero_grad()
            output = _IITnet.forward(torch.Tensor(inputdata))
            loss = loss_fn(output, torch.Tensor(target))
            loss.backward()
            optimizer.step()
        #if epco % 400 == 0:
            #print('epco', epco, 'loss:',loss.item())
    return _IITnet

In [30]:
import time

start = time.time()

trainNet(m)

end = time.time()
print(end-start)

4.8015077114105225


In [104]:
testorders=permutations(m)

energylist = []

for i,ms in zip(range(5),testorders):
    energylist += [ms, [NetEnergy(trainNet(ms)) for j in range(3)]]

print(energylist)

[([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]), [113.74878692626953, 112.23719024658203, 112.02633666992188], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 1], [1, 1, 0]), [99.32850646972656, 101.95207214355469, 92.23348999023438], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 0], [1, 0, 1], [1, 1, 1]), [109.4934310913086, 68.52285766601562, 69.92340850830078], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1], [1, 0, 1]), [98.31291198730469, 112.57115936279297, 122.3484115600586], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 1], [1, 0, 1], [1, 1, 0]), [103.64986419677734, 101.21601867675781, 93.5998306274414]]


In [23]:
NetEnergy(IITnet)

4.904812335968018

In [24]:
def NetEnergy(net):
    totalEnergy = 0.0
    for para in net.parameters():
        flat_para = para.view(para.numel())
        totalEnergy += sum(flat_para * flat_para)
    
    return totalEnergy.item()

In [105]:
len(list(testorders))

40315

In [108]:
m

[[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

In [124]:
check = {}
for order in permutations(m):
    check[str(relabel(m, order))] = relabel(m, order)

In [46]:
def code2str(codeList):
    s = ''
    for i in codeList:
        s += str(i)
    return s

def relabel(m0, order):
    length = len(m0[0])
    bcode = ReverseBinCode(length)
    reorderRule = {}
    for originLabel, newLabel in zip(bcode, order):
        reorderRule[originLabel] = newLabel
    #print(reorderRule)
    rule = {}
    # 初始化rule
    for currentState, nextState in zip(bcode, m0):
        rule[code2str(currentState)] = nextState
    # 更新顺序
    for currentState, nextState in zip(bcode, m0):
        #print(reorderRule[code2str(currentState)], 'to', str(nextState))
        rule[code2str(reorderRule[code2str(currentState)])] = reorderRule[code2str(nextState)]
    
    new_m = [rule[startState] for startState in ReverseBinCode(length)]
    
    return np.array(new_m)

def AllLabelConfigurations(mat):
    # 返回所有可能的重标记方法
    _check = {}
    for order in permutations(mat):
        _check[str(relabel(mat, list(order)))] = relabel(mat, list(order))
    return list(_check.values())

In [47]:
relabel(m,[[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1], [1, 1, 1], [1, 1, 0]])

array([[0, 0, 0],
       [0, 0, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 0]])

In [25]:
len(AllLabelConfigurations(m))

105

In [26]:
all_labels = AllLabelConfigurations(m)

In [161]:
NetEnergy(trainNet(all_labels[0].tolist()))

125.50801086425781

In [31]:
energys = []

# expect time: 16min
start = time.time()
for i, graph in zip(range(5), all_labels):
    energys += [[NetEnergy(trainNet(graph.tolist())) for j in range(40)]]
    
end = time.time()
print(energys)
print('time cost:', end-start)

[[111.5406494140625, 109.67989349365234, 110.9552993774414, 108.75184631347656, 114.32801055908203, 112.37532043457031, 112.61180877685547, 109.82459259033203, 111.62741088867188, 113.04523468017578, 119.81019592285156, 112.51264953613281, 115.0881118774414, 109.69331359863281, 110.81085968017578, 110.8729248046875, 110.69241333007812, 107.08590698242188, 113.44341278076172, 114.39774322509766, 108.85160064697266, 109.6169662475586, 111.40321350097656, 114.26602935791016, 112.2437744140625, 109.07392120361328, 109.16975402832031, 107.95294189453125, 110.02708435058594, 110.11180114746094, 113.12081146240234, 109.57000732421875, 108.73204803466797, 108.83275604248047, 108.76665496826172, 108.86824035644531, 110.09841918945312, 110.09121704101562, 109.96764373779297, 107.59036254882812], [72.22057342529297, 74.84281921386719, 64.8193130493164, 67.5648422241211, 70.52532196044922, 67.50662231445312, 78.20569610595703, 65.0877914428711, 72.6133041381836, 89.07587432861328, 65.6043319702148

# 测试矩阵

In [30]:
m = [[0, 0, 0],
     [0, 0, 1],
     [1, 0, 1],
     [1, 0, 0],
     [1, 1, 0],
     [1, 1, 1],
     [1, 1, 1],
     [1, 1, 0]]

In [4]:
import multiprocessing

In [19]:
core = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=core-1)

In [20]:
import time

start = time.time()

trainNet(m)

nets = pool.map(trainNet, [m for i in range(4)])

end = time.time()
print(end-start)
print(nets)

8.818902015686035
[net(
  (fcn_v): Linear(in_features=3, out_features=3, bias=True)
  (fcn_v2h): Linear(in_features=3, out_features=4, bias=True)
  (fcn_h2v): Linear(in_features=4, out_features=3, bias=True)
), net(
  (fcn_v): Linear(in_features=3, out_features=3, bias=True)
  (fcn_v2h): Linear(in_features=3, out_features=4, bias=True)
  (fcn_h2v): Linear(in_features=4, out_features=3, bias=True)
), net(
  (fcn_v): Linear(in_features=3, out_features=3, bias=True)
  (fcn_v2h): Linear(in_features=3, out_features=4, bias=True)
  (fcn_h2v): Linear(in_features=4, out_features=3, bias=True)
), net(
  (fcn_v): Linear(in_features=3, out_features=3, bias=True)
  (fcn_v2h): Linear(in_features=3, out_features=4, bias=True)
  (fcn_h2v): Linear(in_features=4, out_features=3, bias=True)
)]


In [48]:
all_labels = AllLabelConfigurations(m)

In [49]:
len(all_labels)

1714